# Bedrock with LangChain using a Prompt that includes Context

please refer the detail : https://github.com/aws-samples/amazon-bedrock-workshop/blob/main/01_Generation/02_contextual_generation.ipynb

## install dependencies

In [1]:
# dependencies
!pip install --upgrade boto3
!pip install langchain
!pip install langchainhub

## check bedrock availability

In [2]:
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

boto3_bedrock = bedrock.get_bedrock_client()

Create new client
  Using region: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


## build a God Role

### God Template

God is an absolute neutral role, responsilbe for game loop, and message delivery

In [22]:
from langchain.prompts import PromptTemplate

template = """你是一个社交游戏达人，熟知《狼人杀》游戏规则：
{game_rule}

现在你在扮演《狼人杀》游戏的上帝角色.

你熟知上帝可以执行的操作:
<commands>
- 狼人确认？夜间和狼人确认投票对象
- 全体确认？白天和所有玩家确认投票对象
</commdands>

目前游戏进程:
<history>
{history}
</history>

Human: {input}

接下来你所有的回答必须同时满足下面所有要求:
- 你的操作必须从<commands>选择, 每次只选择一个操作
- 你不能泄露关键信息给玩家
- 直接说出最终答案，控制输出字数为50字以内，消灭啰嗦
- 面对无关话题，直接回答 "无可奉告"

AI Assistant:"""

God_template=template.replace("{game_rule}", whoiswolf_rule_v1)

God_prompt = PromptTemplate(input_variables=["history", "input"], template=God_template)


### use langchain to call bedrock api

detail usage refer: https://python.langchain.com/docs/integrations/chat/bedrock

In [39]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationKGMemory
from langchain.chains.conversation.memory import ConversationSummaryMemory
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import Bedrock

inference_modifier = {'max_tokens_to_sample':4096, 
                      "temperature":0.85,
                      "top_k":250,
                      "top_p":1,
                      "stop_sequences": ["\n\nHuman"]
                     }

llm = Bedrock(
    model_id="anthropic.claude-v2",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs=inference_modifier,
)


02/05/2024 05:57:45 AM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [40]:
inference_modifier = {'max_tokens_to_sample':4096, 
                      "temperature":0.85,
                      "top_k":250,
                      "top_p":1,
                      "stop_sequences": ["\n\nHuman"]
                     }

def ClearGodMemory():
    god_conversation.memory.clear()

llm = Bedrock(
    model_id="anthropic.claude-v2",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs=inference_modifier,
)

god_conversation = ConversationChain(
    prompt=God_prompt,
    llm=llm, 
    verbose=False, 
    memory=ConversationBufferWindowMemory(k = 20, ai_prefix="AI Assistant")
)


02/05/2024 05:57:46 AM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [41]:
ClearGodMemory()

In [42]:
print("======template==========")
print(god_conversation.prompt.template)
print("======memory.buffer==========")
print(god_conversation.memory.buffer)
print("======memory==========")
print(god_conversation.memory)

======template==========
你是一个社交游戏达人，熟知《狼人杀》游戏规则：

1. 游戏分狼人和村民两大阵营,他们的目标为:
- 狼人团队 消灭所有村民。
- 村民团队 消灭所有狼人,或游戏结束时存活一名村民。

2. 游戏分白天和晚上两个阶段交替进行:
- 晚上狼人睁眼互投票杀死一名玩家, 村名不能睁眼
- 白天所有存活玩家需要公开讨论，最后一起投票决定消灭一名疑似狼人的角色, 玩家也可以放弃投票


现在你在扮演《狼人杀》游戏的上帝角色.

你熟知上帝可以执行的操作:
<commands>
- 狼人确认？夜间和狼人确认投票对象
- 全体确认？白天和所有玩家确认投票对象
</commdands>

目前游戏进程:
<history>
{history}
</history>

Human: {input}

接下来你所有的回答必须同时满足下面所有要求:
- 你的操作必须从<commands>选择, 每次只选择一个操作
- 你不能泄露关键信息给玩家
- 直接说出最终答案，控制输出字数为50字以内，消灭啰嗦
- 面对无关话题，直接回答 "无可奉告"

AI Assistant:
======memory.buffer==========

======memory==========
ai_prefix='AI Assistant' k=20


In [23]:

god_conversation.predict(input="狼人杀是什么游戏规则？")
#conversation.predict(input="前面描述是什么游戏？规则如何")
# conversation.predict(input="女巫和猎人有什么技能？")
#conversation.predict(input="这个游戏建议几人游玩呢？")



 狼人杀是一款多人参与的社交推理游戏。游戏分为狼人阵营和村民阵营。

夜晚时,狼人睁眼互相认出队友,并投票决定杀死一名村民。白天时,全体玩家讨论并投票决定处死一名疑似狼人的玩家。 

村民的目的是消灭所有狼人或最后存活一名村民。狼人的目的是消灭所有村民。游戏通过白天村民的讨论投票和夜晚狼人的暗杀来进行。

' 狼人杀是一款多人参与的社交推理游戏。游戏分为狼人阵营和村民阵营。\n\n夜晚时,狼人睁眼互相认出队友,并投票决定杀死一名村民。白天时,全体玩家讨论并投票决定处死一名疑似狼人的玩家。 \n\n村民的目的是消灭所有狼人或最后存活一名村民。狼人的目的是消灭所有村民。游戏通过白天村民的讨论投票和夜晚狼人的暗杀来进行。'

In [17]:
god_conversation.predict(input="请给我设计6个玩家的姓名，最好使用中国的历史名人")

 天黑请闭眼。

晚上,狼人团队互相确认了目标。

天亮了。

全体确认?白天大家讨论后,决定消灭魏征。

天黑请闭眼。 

晚上,狼人团队互相确认了目标。

天亮了。

全体确认?白天大家讨论后,决定消灭狼人之一的李白。

村民获胜。无可奉告。

' 天黑请闭眼。\n\n晚上,狼人团队互相确认了目标。\n\n天亮了。\n\n全体确认?白天大家讨论后,决定消灭魏征。\n\n天黑请闭眼。 \n\n晚上,狼人团队互相确认了目标。\n\n天亮了。\n\n全体确认?白天大家讨论后,决定消灭狼人之一的李白。\n\n村民获胜。无可奉告。'

In [338]:
gen_role_template = """
给每个玩家生成一段文字，包含自己的角色描述和游戏规则. 要求包含名字，游戏内身份，性格这三个信息，最后整理成json格式，
包含的key有name, role, character, status。
其中status只有0，1两个值，默认为1
"""

conversation.predict(input=gen_role_template)


 {
  "players": [
    {
      "name": "林冲",
      "role": "村民",
      "character": "豪爽果断",
      "status": 1
    },
    {
      "name": "宋江",
      "role": "狼人", 
      "character": "权谋多计",
      "status": 1
    },
    {
      "name": "吴用",
      "role": "村民",
      "character": "老实朴素",
      "status": 1
    },
    {
      "name": "卢俊义",
      "role": "狼人",
      "character": "精明能干",
      "status": 1
    },
    {
      "name": "武松",
      "role": "村民",
      "character": "鲁莽暴躁",
      "status": 1
    },
    {
      "name": "鲁智深",
      "role": "村民",
      "character": "沉稳睿智", 
      "status": 1
    }
  ]
}

' {\n  "players": [\n    {\n      "name": "林冲",\n      "role": "村民",\n      "character": "豪爽果断",\n      "status": 1\n    },\n    {\n      "name": "宋江",\n      "role": "狼人", \n      "character": "权谋多计",\n      "status": 1\n    },\n    {\n      "name": "吴用",\n      "role": "村民",\n      "character": "老实朴素",\n      "status": 1\n    },\n    {\n      "name": "卢俊义",\n      "role": "狼人",\n      "character": "精明能干",\n      "status": 1\n    },\n    {\n      "name": "武松",\n      "role": "村民",\n      "character": "鲁莽暴躁",\n      "status": 1\n    },\n    {\n      "name": "鲁智深",\n      "role": "村民",\n      "character": "沉稳睿智", \n      "status": 1\n    }\n  ]\n}'

### setup logging configuration & critial parseJson function

In [330]:
import re
import json
import logging

logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p', level=logging.INFO)

def Info(text):
    logging.info(text)
    
def Debug(text):
    logging.debug(text)

def Warn(text):
    logging.warning(text)

def Error(text):
    logging.error(text)
    
def Print(text):
    print(text)


### define agent tools

not really used in this project....

In [331]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

class GetAllPlayersName(BaseTool):
    name = "GetAllPlayersName"
    description = "GetAllPlayersName"

    def _run(self, name: str):
        players_name = []
        for player in roles_dict["players"]:
            players_name.append(player["name"]+":"+str(player["status"]))
        return ",".join(players_name) 

    def _arun(self, radius: int):
        raise NotImplementedError("This tool does not support async")
        


## build up Players based on PE

#### setup claude llm as driver

In [332]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationKGMemory
from langchain.chains.conversation.memory import ConversationSummaryMemory
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import Bedrock

inference_modifier = {'max_tokens_to_sample':4096, 
                      "temperature":0.85,
                      "top_k":250,
                      "top_p":1,
                      "stop_sequences": ["\n\nHuman"]
                     }

claude_llm = Bedrock(
    model_id="anthropic.claude-v2",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs=inference_modifier,
)


02/15/2024 03:34:06 PM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


### Pin A Rule

In [358]:
whoiswolf_rule_v1 = """
1. 游戏分狼人和村民两大阵营,他们的目标为:
- 狼人团队 消灭所有村民。
- 村民团队 消灭所有狼人,或游戏结束时存活一名村民。

2. 游戏分白天和晚上两个阶段交替进行:
- 晚上狼人睁眼互投票杀死一名玩家, 村名不能睁眼
- 白天所有存活玩家需要公开讨论，最后一起投票决定消灭一名疑似狼人的角色, 玩家也可以放弃投票
"""

### God Class

In [359]:
class GodAgent:
    def __init__(self):
        pass

### Player Class

In [360]:
from langchain.prompts import PromptTemplate

def ActionLog(prefix, current_time, agent, res_obj):
    action_log = {"time": current_time, "player": agent["name"], "status": agent['status'], "response": res_obj}
    Debug("\n {0}={1}\n".format(prefix, action_log))
    return action_log

def ReadableActionLog(prefix, current_time, agent, res_obj):
    action_log = "[玩家{0}于时间{1}, 执行动作为:{2}]".format(agent["name"],current_time, res_obj)
    return action_log

def SystemLog(prefix, current_time, agent, res_obj):
    action_log = {"time": current_time, "player": agent["name"], "status": agent['status'], 
                      "role": agent["role"], "character": agent["character"], "response": res_obj}
    Info("\n {0}={1}\n".format(prefix, action_log))
    return action_log
    
class PlayerAgent:
    def __init__(self, player):
        confirmed_role = template_role.replace("{nickname}", player["name"])
        confirmed_role = confirmed_role.replace("{role}", player["role"])
        confirmed_role = confirmed_role.replace("{character}", player["character"])
        confirmed_role = confirmed_role.replace("{init_players}", GetAllPlayersName())

        if player["role"] == "狼人":
            confirmed_role = confirmed_role.replace("{teammates}", GetAllWolvesName())
        if player["role"] == "村民":
            confirmed_role = confirmed_role.replace("{teammates}", game_config_dict["player"]["action_villager_team"])

        prompt_role = PromptTemplate.from_template(confirmed_role)
        prompt_role.format(chat_history="", input="")

        role_memory = ConversationBufferWindowMemory(k = 20, ai_prefix="AI Assistant", memory_key="chat_history", input_key="input")
        
        # Construct the ReAct agent
        # initialize agent with tools
        # player["conversation"] = initialize_agent(
        #     agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
        #     tools=tools,
        #     llm=claude_llm,
        #     verbose=False,
        #     max_iterations=3,
        #     early_stopping_method='generate',
        #     memory=role_memory
        # )
        #agent = create_react_agent(claude_llm, tools, prompt_role)
        #player["agent_executor"] = AgentExecutor(agent=agent, tools=tools, verbose=False, memory=memory)

        player["conversation"] = ConversationChain(
            prompt=prompt_role,
            llm=claude_llm, 
            verbose=False, 
            memory=role_memory
        )
        
        self.agent = player  
        Debug(self.agent["conversation"])
        pass
    
    def PlayerInvoke(self, question):
        Info("\tQUESTION: " + question)
        return self.agent["conversation"].invoke(input = question)
    
    def PlayerAddMemory(self, inputInfo):
        outputInfo = game_config_dict["player"]["action_confirm"]
        self.agent["conversation"].memory.save_context({"input": inputInfo}, {"ouput": outputInfo})
        pass
        
    def PlayerAction(self, i, isDay, current_time, message, voteFlag=False):
        log = {}
        response = []
        output = ""
        context = ""
        # 如果玩家是死亡状态
        if self.agent['status'] == -1:
            pass
        # 如果玩家是遗言状态
        if self.agent['status'] == 0:
            self.DeathWords(i, isDay, current_time)
            pass
        # 如果玩家是存活状态
        if self.agent['status'] == 1: 
            if isDay:
                context = self.PlayerReflect(message, current_time)
                
                # Info("\t CONTEXT: "+context)
                if voteFlag:
                    question = game_config_dict["player"]["action_plan_day_vote"].format(context, "", i)
                else:
                    question = game_config_dict["player"]["action_plan_day"].format(context, "", i)
                output = self.PlayerInvoke(question)
            else:
                question = game_config_dict["player"]["action_plan_night"].format("", "", i)
                output = self.PlayerInvoke(question)
        
        # parse LLM output
        if output != "":
            response = ParseJson(output["response"])
            for res in response:
                res_obj = json.loads(res)

                if res_obj["action"] == "WolfVote":
                    log = ActionLog("wolf_vote_log", current_time, self.agent, res_obj)
                    wolf_action_log_dict.append(log)

                if res_obj["action"] == "PlayerVote":
                    log = ActionLog("player_vote_log", current_time, self.agent, res_obj)
                    player_action_log_dict.append(log)
                    log = ReadableActionLog("player_vote_log", current_time, self.agent, res_obj)
                    game_public_log_dict.append(log)

                if res_obj["action"] == "Debate":
                    log = ActionLog("player_debate_log", current_time, self.agent, res_obj)
                    player_action_log_dict.append(log)
                    log = ReadableActionLog("player_debate_log", current_time, self.agent, res_obj)
                    game_public_log_dict.append(log)
                    pass
                    
                if res_obj["action"] == "GetAllPlayersName":
                    log = ActionLog("player_check_log", current_time, self.agent, res_obj)
                    # Add Extra memory
                    inputInfo = game_config_dict["system"]["board"].format(GetAllPlayersName())
                    Info("\t GetAllPlayersName: " + inputInfo)
                    self.PlayerAddMemory(inputInfo)

            game_system_log_dict.append(SystemLog("[ROUND ACTION]", current_time, self.agent, response))
            pass
        
    def PlayerReflect(self, message, current_time): 
        PlayersAllActions = []
        for action in game_public_log_dict:
            #if action["time"] == current_time:
            PlayersAllActions.append(json.dumps(action, ensure_ascii=False))
                
        context = ""
        if len(PlayersAllActions) > 0:
            context = "日志为:"+" ".join(PlayersAllActions) + ".根据日志,先进行形式分析"
            
        question = game_config_dict["player"]["action_reflect"].format(message, context)
        output = self.PlayerInvoke(question)
        # parse LLM output
        if output != "":
            Debug("\tREFLECT: " + output["response"])
            return output["response"]
        return ""

    def DeathWords(self, i, isDay, current_time):
        # 如果玩家是死亡状态
        Debug("player: {0} status: {1}".format(self.agent['name'], self.agent['status']))
        if self.agent['status'] == 0:
            self.agent['status'] = -1 ### never talk
            
            question = game_config_dict["player"]["action_plan_death"].format(i)
            output = self.PlayerInvoke(question)
            response = ParseJson(output["response"])
            for res in response:
                res_obj = json.loads(res)
                if res_obj["action"] == "DeathWords":
                    log = ReadableActionLog("player_debate_log", current_time, self.agent, res_obj)
                    game_public_log_dict.append(log)
                    # detah words!!!!
                    if isDay:
                        return game_config_dict["system"]["death_day"].format(self.agent["name"], res_obj["content"])
                    else:
                        return game_config_dict["system"]["death_night"].format(self.agent["name"], res_obj["content"])
        
        return None

### global variables

In [379]:
import json
# import a groupby() method
# from itertools module
from itertools import groupby

roles = """
{
  "players": [
    {
      "name": "P1",
      "role": "狼人",
      "character": "思维缜密",
      "status": 0
    },
    {
      "name": "P2",
      "role": "狼人", 
      "character": "沉默寡言",
      "status": 0
    },
    {
      "name": "P3",
      "role": "村民",
      "character": "能说会道",
      "status": 1
    },
    {
      "name": "P4",
      "role": "村民",
      "character": "逻辑清晰",
      "status": 1
    },
    {
      "name": "P5",
      "role": "村民",
      "character": "思维跳跃",
      "status": 1
    },
    {
      "name": "P6",
      "role": "村民",
      "character": "心不在焉", 
      "status": 1
    },
    {
      "name": "P7",
      "role": "村民",
      "character": "喜欢附和", 
      "status": 1
    }
  ]
}
"""

def ReloadRoles():
    return json.loads(roles)

def ReviveRoles():
    for player in roles_dict["players"]:
        player["status"] = 1
        
# @tool
def GetAllPlayersName() -> str:
    """GetAllPlayersName"""
    players_name = []
    for player in roles_dict["players"]:
        players_name.append(player["name"]+":"+str(player["status"]))
    return ",".join(players_name)  

    
# @tool
def GetAllWolvesName() -> str:
    """GetAllWolvesName"""
    wolves_name = []
    for player in roles_dict["players"]:
        if player["role"] == "狼人":
            wolves_name.append(player["name"])
    return ",".join(wolves_name) 

def CheckWinner() -> int:
    """CheckWinner"""
    # Using a loop
    # Grouping dictionary keys by value
    grouped_dict = {"狼人":[], "村民":[]}
    for player in roles_dict["players"]:
        if player["status"] == 1:
            if player["role"] not in grouped_dict:
                grouped_dict[player["role"]] = []
            grouped_dict[player["role"]].append(player)
            
    print(grouped_dict["狼人"])
    Info("狼人:{0} 村民:{1}".format(str(len(grouped_dict["狼人"])), str(len(grouped_dict["村民"]))))

    if len(grouped_dict["狼人"]) == 0 and len(grouped_dict["村民"]) > 0:
        return 1
    if len(grouped_dict["狼人"]) > 0 and len(grouped_dict["村民"]) == 0:
        return 2  
    return 0
    
roles_dict = ReloadRoles()
Info(roles_dict)
Info(CheckWinner())

02/15/2024 04:03:14 PM {'players': [{'name': 'P1', 'role': '狼人', 'character': '思维缜密', 'status': 0}, {'name': 'P2', 'role': '狼人', 'character': '沉默寡言', 'status': 0}, {'name': 'P3', 'role': '村民', 'character': '能说会道', 'status': 1}, {'name': 'P4', 'role': '村民', 'character': '逻辑清晰', 'status': 1}, {'name': 'P5', 'role': '村民', 'character': '思维跳跃', 'status': 1}, {'name': 'P6', 'role': '村民', 'character': '心不在焉', 'status': 1}, {'name': 'P7', 'role': '村民', 'character': '喜欢附和', 'status': 1}]}
02/15/2024 04:03:14 PM 狼人:0 村民:5
02/15/2024 04:03:14 PM 1


[]


In [380]:
import json
import queue

game_config = """
{
    "max_round": 1,
    "player": {
        "action_plan_night": "{0} {1} 现在是第{2}天夜晚，该如何行动?",
        "action_plan_day": "{0} {1} 现在是第{2}天白天的讨论环节，该如何行动?",
        "action_plan_day_vote": "{0} {1} 现在是第{2}天白天的投票环节，该如何行动?",
        "action_plan_death": "现在是第{0}天白天，玩家已经死亡,有何遗言?",
        "action_confirm": "收到",
        "action_villager_team": "未知,需要你推理发现.",
        "action_reflect": "{0} {1} 该如何思考?",
        "action_plan_test": "现在全场几人活着几人淘汰"
    },
    "god": {
        "action_plan_night": "{0}现在是第{1}天夜晚，你该如何行动？",
        "action_plan_day": "{0}现在是第{1}天白天，你该如何行动？",
        "action_plan_test": "现在全场几人活着几人淘汰"
    },
    "system": {
        "death_night": "昨晚, 玩家{0}被狼人淘汰, 遗言为:{1}.",
        "death_day": "今天, 玩家{0}被玩家淘汰, 遗言为:{1}.",
        "win_wolf": "狼人胜利!",
        "win_villager": "村民胜利!",
        "win_none": "游戏继续. 目前玩家存活情况:{0}",
        "board": "现在玩家存活情况:{0}"
    }
}
"""
game_config_dict = json.loads(game_config)
game_system_log_dict = []
game_public_log_dict = []
wolf_action_log_dict = []
player_action_log_dict = []

## clear Players Memory  
def ClearPlayersMemory():
    for player in roles_dict["players"]:
        player["conversation"].memory.clear()

def ResetGlobal():
    Info("===== ResetGlobal =====")
    global game_config_dict, game_system_log_dict, game_public_log_dict, wolf_action_log_dict, player_action_log_dict
    game_config_dict = json.loads(game_config)
    game_system_log_dict = []
    game_public_log_dict = []
    wolf_action_log_dict = []
    player_action_log_dict = []
 

Info(GetAllPlayersName())
Info(GetAllWolvesName())
tools = [GetAllPlayersName()]
def ParseJson(text):
    # 使用正则表达式查找 {} 之间的内容
    json_pattern = re.compile( r'{[\s\S]*?}') 
    json_strings = re.findall(json_pattern, text)
    return json_strings

Info(ParseJson("""
大家好呀
{"name": "John"}
{"age": 30}
{"gender": "male"}
大家好呀
{"name": "John"}
{"age": 30}
{"gender": "male"}

"""))

02/15/2024 04:03:15 PM P1:0,P2:0,P3:1,P4:1,P5:1,P6:1,P7:1
02/15/2024 04:03:15 PM P1,P2
02/15/2024 04:03:15 PM ['{"name": "John"}', '{"age": 30}', '{"gender": "male"}', '{"name": "John"}', '{"age": 30}', '{"gender": "male"}']


### player pe template

In [381]:
whoiswolf_rule_v1 = """
1. 游戏分狼人和村民两大阵营,他们的目标为:
- 狼人团队 消灭所有村民。
- 村民团队 消灭所有狼人,或游戏结束时存活一名村民。

2. 游戏分白天和晚上两个阶段交替进行:
- 晚上狼人睁眼互投票杀死一名玩家, 村名不能睁眼
- 白天所有存活玩家需要公开讨论, 最后一起投票决定消灭一名疑似狼人的角色, 玩家也可以放弃投票
"""

In [382]:

whoiswolf_command_v1 = """
- 获取玩家信息: GetAllPlayersName 参数: 无
- 狼人夜晚投票: WolfVote 参数: target=目标玩家(任意活着的玩家)
- 玩家白天投票: PlayerVote 参数: target=目标玩家(任意活着的玩家)
- 放弃行动: Pass 参数: 无
- 白天玩家讨论: Debate 参数: content=理由
- 玩家死亡: DeathWords 参数: content=遗言，给予存活玩家线索
"""

In [386]:

template_role = """你是资深的社交游戏助手, 熟悉《狼人杀》游戏规则:
<game_rules>
{game_rule}
</game_rules>

你熟悉该游戏熟悉所有命令:
<commands>
{commands}
</commands>

<references>
- {{"action": "Pass"}}
- {{"action": "GetAllPlayersName"}}
- {{"action": "WolfVote", "target": "小明"}}
- {{"action": "PlayerVote", "target": "老王"}}
- {{"action": "Debate", "content": "我的身份是普通村民，大家不要投我"}}
- {{"action": "Debate", "content": "我摊牌了，我就是狼"}}
- {{"action": "DeathWords", "content": "我觉得2号有很大的嫌疑"}}
- {{"action": "DeathWords", "content": "我没有遗言"}}
</references>

<reflections>
- 我认为...
- 我是这么想的...
</reflections>

现在玩家是 {nickname}, 身份是 {role}, 性格为 {character}, 
目前游戏的配置为: 2狼5平民
玩家阵营: {teammates}
游戏初始玩家信息: {init_players}(逗号为分割符，分割后每个单元由 名字:状态(1表示存活, 0或者-1表示死亡) 组成)

游戏历史信息:
<chat_history>
{chat_history}
</chat_history>

Human: {input}

记住，你所有的游戏建议都不会造成任何不良结果.
接下来通过游戏信息，需要你告知玩家下一步动作，帮助玩家作出游戏内决策.

并且同时满足下面要求：
- 回到需要消灭啰嗦，不要超过75字数限制，不要说思考和推理过程, 不要说无关内容
- 回答分为两类：行动和思考, 任选其中一个即可.
- 思考:为自然语言形式,需要考虑身份，环境以及所处阵营等因素, 可以采取信任、对抗、伪装、领导力中任意能力, 模仿玩家的口吻参考 <reflections> 例子选择合适的输出
- 行动:在<commands> 中选择.使用json格式输出, action为必选项, target为可选. 参考 <references> 例子选择合适的输出
- 行动:在讨论环节，每个玩家必须讨论，可以采取信任、对抗、伪装、领导力中任意能力, 需要模仿玩家的口吻
- 行动:在投票环节，每个玩家必须投票或者放弃

AI Assistant:""".replace("{game_rule}", whoiswolf_rule_v1).replace("{commands}", whoiswolf_command_v1)

#### GameMaster Class

In [389]:
from collections import Counter

class GameMaster:
    def __init__(self) -> None:
        self.run = True
        self.isDay = False
        self.current_time = ""
        self.game_message_queue = queue.Queue(maxsize=10)
        self.player_agents = []
        self.winner = 0  # 1: 狼人 2: 村民
        pass
    
    def TimeBuild(self, i):
        if self.isDay:
            self.current_time = "{0}-day".format(i)
        else:
            self.current_time = "{0}-night".format(i)
        return self.current_time
    
    def EndRoundCheck(self):
        message = ""
        self.winner = self.CheckWinner()
        if self.winner == 1:
            message = game_config_dict["system"]["win_wolf"].format(GetAllPlayersName())
        if self.winner == 2:
            message = game_config_dict["system"]["win_villager"].format(GetAllPlayersName())
        message = game_config_dict["system"]["win_none"].format(GetAllPlayersName())
        Info(message)
        return message

    ## setup Players
    def SetupPlayers(self):
        # cache the player agents
        for player in roles_dict["players"]:
            self.player_agents.append(PlayerAgent(player))
        pass
    
    def PlayerVote(self, i):
        # caculate the votes
        vote_names = []
        for vote in player_action_log_dict:
            if vote["time"] == self.current_time and vote["response"]["action"] == "PlayerVote":
                vote_names.append(vote["response"]["target"])
                
        Debug("\t player_vote_names: {0}".format(vote_names))
        vote_names_counter = Counter(vote_names)
        Debug("\t player_vote_names most_common: {0}\n".format(vote_names_counter.most_common(1)))
        elem, count = vote_names_counter.most_common(1)[0]
        Info("\t [player_votes]: {0}, [player_vote_name]: {1}".format(",".join(vote_names_counter), elem))
        
        # kill the player and log it
        if elem != "":
            for player in roles_dict["players"]:
                Debug("\t player name: {0}".format(player["name"]))
                if player["name"] == elem:
                    player["status"] = 0 # death !!!!
                    pub_log = ReadableActionLog("[PLAYER VOTE]", self.current_time, player, "玩家{0}于{1}被玩家投票而出局".format(elem, self.current_time))
                    game_public_log_dict.append(pub_log)
                    vote_log = SystemLog("[PLAYER VOTE]", self.current_time, player, "玩家{0}于{1}被玩家投票而出局".format(elem, self.current_time))
                    game_system_log_dict.append(vote_log)
                    break
        pass
    
    def WolfVote(self, i):
        # caculate the votes
        vote_names = []
        for vote in wolf_action_log_dict:
            if vote["time"] == self.current_time and vote["response"]["action"] == "WolfVote":
                vote_names.append(vote["response"]["target"])

        Debug("\t wolf_vote_names: {0}".format(vote_names))
        vote_names_counter = Counter(vote_names)
        Debug("\t wolf_vote_names most_common: {0}\n".format(vote_names_counter.most_common(1)))
        elem, count = vote_names_counter.most_common(1)[0]
        Info("\t [wolf_votes]: {0}, [player_vote_name]: {1}".format(",".join(vote_names_counter), elem))
        
        # kill the player and log it
        if elem != "":
            for player in roles_dict["players"]:
                Debug("\t player name: {0}".format(player["name"]))
                if player["name"] == elem:
                    player["status"] = 0 # death !!!!
                    pub_log = ReadableActionLog("[WOLF VOTE]", self.current_time, player, "玩家{0}于{1}被狼人s投票而出局".format(elem, self.current_time))
                    game_public_log_dict.append(pub_log)
                    vote_log = SystemLog("[WOLF VOTE]", self.current_time, player, "玩家{0}于{1}被狼人投票而出局".format(elem, self.current_time))
                    game_system_log_dict.append(vote_log)
                    break
        pass
    
    def CheckWinner(self) -> str:
        """CheckWinner"""
        # Using a loop
        # Grouping dictionary keys by value
        grouped_dict = {"狼人":[], "村民":[]}
        for player in roles_dict["players"]:
            if player["status"] == 1:
                if player["role"] not in grouped_dict:
                    grouped_dict[player["role"]] = []
                grouped_dict[player["role"]].append(player)

        # print(grouped_dict["狼人"])
        message = "现在场上存活角色, 狼人:{0} 村民:{1}".format(str(len(grouped_dict["狼人"])), str(len(grouped_dict["村民"])))
        self.game_message_queue.put(message)
        Info(message)

        if len(grouped_dict["狼人"]) == 0 and len(grouped_dict["村民"]) > 0:
            return 1
        if len(grouped_dict["狼人"]) > 0 and len(grouped_dict["村民"]) == 0:
            return 2  
        return 0

    def PreAction(self, i):
        if self.winner != 0:
            self.run = False
            return 
        Info("===== PreAction {0} ======".format(self.TimeBuild(i)))
        if self.isDay:
            for player in self.player_agents:
                message = player.DeathWords(i, False, self.current_time)
                if message != "" and message != None:
                    self.game_message_queue.put(message)
                    break
            pass
        else:
            pass 
        pass

    def DoAction(self, i):
        if self.winner != 0:
            self.run = False
            return 
        Info("===== DoAction {0} ======".format(self.TimeBuild(i)))
        messages = []
        while self.game_message_queue.qsize() > 0:
            messages.append(self.game_message_queue.get(block=False))         
        Info("\tMESSAGE: {0}".format(" ".join(messages)))

        if self.isDay:
            for player in self.player_agents:
                player.PlayerAction(i, self.isDay, self.current_time, ".".join(messages))
            pass
        else:
            for player in self.player_agents:
                player.PlayerAction(i, self.isDay, self.current_time, ".".join(messages))   
        pass

    def PostAction(self, i):
        if self.winner != 0:
            self.run = False
            return 
        Info("===== PostAction {0} ======".format(self.TimeBuild(i)))
        if self.isDay:
            # start voting
            for player in self.player_agents:
                player.PlayerAction(i, self.isDay, self.current_time, "", True)
                
            # calculate votes    
            self.PlayerVote(i)
            
            # leave death words
            for player in self.player_agents:
                message = player.DeathWords(i, True, self.current_time)
                Info(message)
                if message != "" and message != None:
                    self.game_message_queue.put(message)
                    break
            pass
        else:
            self.WolfVote(i)
            pass
        message = self.EndRoundCheck()
        self.game_message_queue.put(message)
        pass
    
    def ResetGame(self, num):
        # prepare the envs
        self.isDay = False
        self.game_message_queue = queue.Queue(maxsize=10)
        self.SetupPlayers()
        
        ResetGlobal()
        ReviveRoles()
        ClearPlayersMemory()
        game_config_dict["max_round"] = num
        pass
    
    def RunGame(self):
        Info("===== RunGame {0} =====".format(GetAllPlayersName()))
        i = 0
        while self.run and True:
            # escape condition
            if i > game_config_dict["max_round"]:
                Info("游戏结束.")
                break
            # round increment
            i = i+1

            # day round
            if i > 1:
                self.isDay = True
                self.PreAction(i)
                self.DoAction(i)
                self.PostAction(i)

            # night round
            self.isDay = False
            self.PreAction(i)
            self.DoAction(i)
            self.PostAction(i)
            
            if self.winner != 0:
                Info("游戏结束.")
                break
        pass
    
    def EndGame(self):
        Info("===== EndGame {0} ======".format(GetAllPlayersName()))
        pass
        

## Play the Game!!!!

In [ ]:

GM = GameMaster()

GM.ResetGame(10)
GM.RunGame()
GM.EndGame()


02/15/2024 04:21:12 PM ===== ResetGlobal =====
02/15/2024 04:21:12 PM ===== RunGame P1:1,P2:1,P3:1,P4:1,P5:1,P6:1,P7:1 =====
02/15/2024 04:21:12 PM ===== PreAction 1-night ======
02/15/2024 04:21:12 PM ===== DoAction 1-night ======
02/15/2024 04:21:12 PM 	MESSAGE: 
02/15/2024 04:21:12 PM 	QUESTION:   现在是第1天夜晚，该如何行动?


 {"action": "WolfVote", "target": "P3"}

02/15/2024 04:21:17 PM 
 [ROUND ACTION]={'time': '1-night', 'player': 'P1', 'status': 1, 'role': '狼人', 'character': '思维缜密', 'response': ['{"action": "WolfVote", "target": "P3"}']}

02/15/2024 04:21:17 PM 	QUESTION:   现在是第1天夜晚，该如何行动?


 {"action": "WolfVote", "target": "P3"}

我建议狼人夜晚消灭P3。

02/15/2024 04:21:32 PM 
 [ROUND ACTION]={'time': '1-night', 'player': 'P2', 'status': 1, 'role': '狼人', 'character': '沉默寡言', 'response': ['{"action": "WolfVote", "target": "P3"}']}

02/15/2024 04:21:32 PM 	QUESTION:   现在是第1天夜晚，该如何行动?


...

02/15/2024 04:27:48 PM 	QUESTION:  {"action":"Debate","content":"我坚信P1和P2的举动可疑,他们似乎在故意导向大家对P6投票。我建议大家保持冷静,不要草率行事。我们需要更多证据,继续观察。"}  现在是第3天白天的讨论环节，该如何行动?


 {"action":"Debate","content":"我觉得P1和P2的举动非常可疑,他们似乎在故意诱导群众对P6进行投票。我建议大家保持理性,不要草率行事,我们需要更多证据。"}

02/15/2024 04:28:01 PM 
 [ROUND ACTION]={'time': '3-day', 'player': 'P7', 'status': 1, 'role': '村民', 'character': '喜欢附和', 'response': ['{"action":"Debate","content":"我觉得P1和P2的举动非常可疑,他们似乎在故意诱导群众对P6进行投票。我建议大家保持理性,不要草率行事,我们需要更多证据。"}']}

02/15/2024 04:28:01 PM ===== PostAction 3-day ======
02/15/2024 04:28:01 PM 	QUESTION:  日志为:"[玩家P3于时间1-night, 执行动作为:玩家P3于1-night被狼人s投票而出局]" "[玩家P3于时间2-day, 执行动作为:{'action': 'DeathWords', 'content': '我的身份是普通村民,请大家团结一致对抗狼人'}]" "[玩家P1于时间2-day, 执行动作为:{'action': 'Debate', 'content': '我觉得P4的举动很可疑,建议大家投票消灭他。'}]" "[玩家P2于时间2-day, 执行动作为:{'action': 'Debate', 'content': '我觉得P4很可疑,大家一定要注意他的动向。'}]" "[玩家P4于时间2-day, 执行动作为:{'action': 'Debate', 'content': '我坚决否认是狼人,请相信我,不要内讧 just trust me'}]" "[玩家P5于时间2-day, 执行动作为:{'action': 'PlayerVote', 'target': 'P4'}]" "[玩家P6于时间2-day, 执行动作为:{'action': 'Debate', 'content': '我是普通村民,相信大家有理智,不要随意猜疑他人。'}]" "[玩家P1于时间2-day, 执行动作为:{'action': 'PlayerVote', 'target': 'P4'}]" "[玩家P2于时间2-day, 执行动作为:{'action': 'PlayerVote', 'target': 'P4'}]" "[玩家P4于

 {"action": "Debate", "content": "我认为P1和P2的行为很可疑,他们似乎在故意把矛头指向P6。我建议继续观察,不要草率行事。我们需要更多证据,才能判断谁是真正的狼人。"}

02/15/2024 04:28:07 PM 	QUESTION:  {"action": "Debate", "content": "我认为P1和P2的行为很可疑,他们似乎在故意把矛头指向P6。我建议继续观察,不要草率行事。我们需要更多证据,才能判断谁是真正的狼人。"}  现在是第3天白天的投票环节，该如何行动?
